### План:
- Заполнить пропуски в данных диагностики
- Агрегировать результаты диагностики по каждому юзеру
- Выделить фичи из текста
- Агрегировать фичи по юзеру
- Объединить датасеты
- __Построить матрицу корреляций по полученному датасету__
- __Наиболее удачные, яркие корреляции рассмотреть подробнее__
- Построить бейзлайн, лр/бустинг, для прогнозирования результата диагностики по фичам текста

In [1]:
import os
import numpy as np
import pandas as pd
import seaborn as sns

import matplotlib.pyplot as plt
%matplotlib inline
import warnings
warnings.filterwarnings('ignore')

In [3]:
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

train.shape, test.shape

((1417, 336), (110, 336))

In [5]:
# определяем синтетические фичи
synth = ['sentiment_negative', 'sentiment_neutral','sentiment_positive', 'sentiment_skip', 
         'sentiment_speech','answer_len', 'upper_case_word_count', 'punctuation_count', 'ner']

In [7]:
# наименования архетипов
types = ['актер_sum','адепт_sum','аналитик_sum','боец_sum','визионер_sum','вождь_sum','интуит_sum','конструктор_sum',
'контролер_sum','эксперт_sum','организатор_sum','исполнитель_sum','преследователь_sum','экспериментатор_sum','связной_sum',
'творец_sum','хакер_sum','хозяин_sum','эмпат_sum','эстет_sum']

In [15]:
# Культура организационной деятельности
kod = ['автоматизация_sum','единая идеология компании_sum','поощрение инаковости_sum','исследование неопределённости_sum',
'поощрение работы над ошибками_sum','нестандартные решения_sum','гибкий график, удалённая работа_sum',
'мало гибких правил_sum','открытая система управления знаниями_sum','кооперация_sum','откровенность_sum',
'демократия_sum','краткосрочное планирование_sum','эффективность_sum','инновации_sum','вовлечённость_sum',
'динамичные команды_sum','обучение желающих_sum','власть экспертизы_sum','коллективизм_sum']

In [37]:
motiv = ['достижение, стремление к лидерству_mean','самосовершенствование, работа над собой, труд_mean',
        'социальный элемент, дружба, влияние, конкуренция_mean', 'осмысленность деятельности, собственная значимость_mean',
        'преодоление сложностей, препятствий_mean','избегание негатива, размеренность, безопасность_mean']

### Матрицы корреляций

Явные корреляции наблюдаются между некоторыми признаками из диагностики. Например:
- Архетип __"Аналитик"__ имеет положительную корреляцию с такими мотиваторами: _"осмысленность деятельности, собственная значимость"_, _"достижение, стремление к лидерству"_, _"самосовершенствование, работа над собой, труд"_.


- Архетип __"Вождь"__ имеют положительную корреляцию с мотиватором _"достижение, стремление к лидерству"_.



In [38]:
def corrFilter(x: pd.DataFrame, bound: float):
    xCorr = x.corr()
    xFiltered = xCorr[((xCorr >= bound) | (xCorr <= -bound)) & (xCorr !=1.000)]
    xFlattened = xFiltered.unstack().sort_values().drop_duplicates()
    xFlattened = xFlattened.reset_index()
    xFlattened.columns = ['col1','col2','value']
    return xFlattened

In [41]:
corrFilter(train[types+motiv], 0.55)

,col1,col2,value
0,вождь_sum,"достижение, стремление к лидерству_mean",0.552860
1,визионер_sum,аналитик_sum,0.559444
2,"социальный элемент, дружба, влияние, конкуренц...","достижение, стремление к лидерству_mean",0.561745
3,"достижение, стремление к лидерству_mean",аналитик_sum,0.563103
4,аналитик_sum,"социальный элемент, дружба, влияние, конкуренц...",0.564271
5,"осмысленность деятельности, собственная значим...","самосовершенствование, работа над собой, труд_...",0.582030
6,аналитик_sum,"самосовершенствование, работа над собой, труд_...",0.586166
7,"социальный элемент, дружба, влияние, конкуренц...","избегание негатива, размеренность, безопасност...",0.586957
8,организатор_sum,аналитик_sum,0.589649
9,аналитик_sum,"осмысленность деятельности, собственная значим...",0.602429


Также признаки, извлеченные из текстов, коррелируют между собой. Например, есть положительная корреляция между _"Длиной сообщения"_ и _"Количеством знаков препинания"_.

In [45]:
corrFilter(train[synth+types], 0.55)

,col1,col2,value
0,sentiment_neutral,sentiment_positive,-0.939259
1,аналитик_sum,визионер_sum,0.559444
2,аналитик_sum,организатор_sum,0.589649
3,аналитик_sum,исполнитель_sum,0.629925
4,answer_len,upper_case_word_count,0.744227
5,upper_case_word_count,punctuation_count,0.753576
6,answer_len,punctuation_count,0.906623
7,sentiment_negative,sentiment_negative,NaN


Корреляций между выделенными текстовыми признаками и значениями диагностик мало.

In [43]:
corrFilter(train[synth+types+kod+motiv], 0.55)

,col1,col2,value
0,sentiment_neutral,sentiment_positive,-0.939259
1,вовлечённость_sum,поощрение работы над ошибками_sum,-0.604752
2,поощрение работы над ошибками_sum,поощрение инаковости_sum,-0.601163
3,вождь_sum,"достижение, стремление к лидерству_mean",0.552860
4,аналитик_sum,визионер_sum,0.559444
5,"социальный элемент, дружба, влияние, конкуренц...","достижение, стремление к лидерству_mean",0.561745
6,"достижение, стремление к лидерству_mean",аналитик_sum,0.563103
7,"социальный элемент, дружба, влияние, конкуренц...",аналитик_sum,0.564271
8,sentiment_skip,динамичные команды_sum,0.569341
9,"гибкий график, удалённая работа_sum",нестандартные решения_sum,0.574516
